# Optimizing Ad Delivery with `pi_optimal`

## Introduction

The managment of digital advertising campaigns is a complex and dynamic process, and still requires often human work to optimize the delivery of ads to maximize the desired outcome, such as smooth and evenly distributed impression delivery over time. By leveraging **Reinforcement Learning (RL)**, we can optimize the delivery of ads to support or automate human campaign managment. This notebook demonstrates the use of `pi_optimal` to train an RL agent for **ad delivery optimization**, utilizing a dataset of historical ad data, about the campiagns delevery as well as the campaign manager's actions.

---

## Problem Statement

We aim to automate and optimize campaign management for a marketing agency specializing in regional online advertising. Due to the limited geographical scope, achieving smooth and consistent ad delivery over time presents a significant challenge. The agency has provided a dataset of historical ad delivery data. Leveraging `pi_optimal`, our goal is to train a reinforcement learning (RL) agent capable of:

- **Delivering ads in a smooth, linear, and efficient manner** throughout the entire campaign duration. 

This approach seeks to enhance campaign performance, ensure better resource utilization, and improve audience targeting within regional constraints.

---

## Dataset

The dataset contains 57 adsets which data was collected between the 15th of July 2024 and 13th of November 2024. Every hour a snapshot of the adset state was taken, including the number of impressions delivered by the adset. The dataset also includes the actions taken by the campaign manager to adjust the adset delivery.

The dataset contains the following columns:

### Dataset Features

1. **AdSet General Information**:
    - `created_at`: The date and time where the state and actions of the adset was recorded.
    - `unit_index`: The unique identifier of the adset.
    - `hour_of_day`: The hour of the day when the adset datapoint was recorded.
    - `day_of_week`: The day of the week when the adset datapoint was recorded.
    - `adset_name`: The name of the adset.

2. **AdSet Delivery Information**:
    - `adset_impressions_diff`: The number of impressions delivered in this hour.
    - `adset_settings_total_budget`: The total budget of the adset.
    - `adset_settings_daily_budget`: The daily budget of the adset.
    - `adset_settings_maximum_daily_impressions`: The maximum number of impressions the adset can deliver in a day.
    - `adset_settings_maximum_lifetime_impressions`: The maximum number of impressions the adset can deliver in its lifetime.
    - `adset_settings_runtime_in_hours`: The total number of hours the adset is going to run.
    - `adset_settings_remaining_hours`: The remaining number of hours the adset will run.
    - `adset_settings_running_hours`: The number of hours the adset has been running.

3. **Campaign Manager Actions**:
    - `adset_settings_maximum_cpm`: The maximum CPM set for a specific hour by the campaign manager.
    - `adset_targetings_frequency_capping_requests`: The frequency capping requests used by the campaign manager. Frequency capping is a feature that limits the number of times an ad is shown to the same user.
    - `adset_settings_bidding_strategy`: The bidding strategy method.
    - `adset_settings_pacing_type`: The pacing type of the campaign. Pacing is the rate at which the adset spends its budget.


4. **AdSet Delivery Metrics**:
    - `total_missing_impressions`: The total number of missing impressions of the total number of impression.
    - `expected_impression_next_hour`: The expected number of impressions the adset will deliver in the next hour.
    - `hourly_missing_impressions`: The number of missing impressions in this hour.

In [1]:
import pandas as pd

df_historical_adset_control = pd.read_csv('data/historical_adset_control.csv', parse_dates=['created_at'])
df_historical_adset_control.head()

,created_at,unit_index,hour_of_day,day_of_week,adset_name,adset_impressions,adset_impressions_diff,adset_settings_total_budget,adset_settings_daily_budget,adset_settings_maximum_daily_impressions,adset_settings_maximum_lifetime_impressions,adset_settings_runtime_in_hours,adset_settings_remaining_hours,adset_settings_running_hours,adset_targetings_total_population,adset_settings_maximum_cpm,adset_targetings_frequency_capping_requests,adset_settings_bidding_strategy,adset_settings_pacing_type
0,2024-07-15 14:04:55.531,0,14,Monday,Billboard,72385.0,0.0,295120,21080,18068.0,252960.0,336,247,88,1439387.0,1190,10.0,IntervalBudgetOptimized,Off
1,2024-07-15 14:04:55.531,1,14,Monday,AdBundle,288275.0,7.0,272250,19446,72128.0,1009800.0,336,247,88,1439387.0,275,10.0,IntervalBudgetOptimized,Off
2,2024-07-15 18:04:51.084,1,18,Monday,AdBundle,288282.0,7.0,272250,19446,72128.0,1009800.0,336,243,92,1439387.0,275,10.0,IntervalBudgetOptimized,Off
3,2024-07-15 18:04:51.084,0,18,Monday,Billboard,72385.0,0.0,295120,21080,18068.0,252960.0,336,243,92,1439387.0,1190,10.0,IntervalBudgetOptimized,Off
4,2024-07-15 19:05:02.736,1,19,Monday,AdBundle,288282.0,0.0,272250,19446,72128.0,1009800.0,336,242,93,1439387.0,275,10.0,IntervalBudgetOptimized,Off


In [2]:
# Uncomment the following lines for faster training and inference if you have sklearnex installed and are using an Intel CPU

#import numpy as np
#from sklearnex import patch_sklearn
#patch_sklearn()

In [3]:
# Add the root path to the sys path to load pi_optimal from the parent directory
import sys
sys.path.append("../..")

---

## Defining the Reward Function

The reward function is quite simple and just wants to minimize the hourly missing impressions. The reward function is defined as follows:

$\text{Reward} = - (\text{Hourly Missing Impressions})$

### Implementation

In [4]:
# Function to calculate reward
def calculate_reward(row, epsilon=1e-8):

    # Calculate the total missing impressions
    total_missing_impressions = row.adset_settings_maximum_lifetime_impressions - row.adset_impressions

    # Calculate the expected impressions in the next hour (epsilon is added to avoid division by zero)
    expected_impression_next_hour = (total_missing_impressions / (row.adset_settings_remaining_hours + epsilon))
    expected_impression_next_hour = max(0, expected_impression_next_hour)

    # Calculate the hourly missing impressions which prepresents the performance of the current control settings
    # when a lot of impressions are missing, the control settings were not optimal
    hourly_missing_impressions = ((expected_impression_next_hour - row.adset_impressions_diff) ** 2) ** 0.5

    # Total penalty
    total_penalty = hourly_missing_impressions

    # Reward is the negative of the total penalty
    reward = -total_penalty
    return reward

### Apply the reward function to the dataset

In [5]:
# Apply the reward calculation
df_historical_adset_control['reward'] = df_historical_adset_control.apply(calculate_reward, axis=1)
df_historical_adset_control.head()

,created_at,unit_index,hour_of_day,day_of_week,adset_name,adset_impressions,adset_impressions_diff,adset_settings_total_budget,adset_settings_daily_budget,adset_settings_maximum_daily_impressions,adset_settings_maximum_lifetime_impressions,adset_settings_runtime_in_hours,adset_settings_remaining_hours,adset_settings_running_hours,adset_targetings_total_population,adset_settings_maximum_cpm,adset_targetings_frequency_capping_requests,adset_settings_bidding_strategy,adset_settings_pacing_type,reward
0,2024-07-15 14:04:55.531,0,14,Monday,Billboard,72385.0,0.0,295120,21080,18068.0,252960.0,336,247,88,1439387.0,1190,10.0,IntervalBudgetOptimized,Off,-731.072874
1,2024-07-15 14:04:55.531,1,14,Monday,AdBundle,288275.0,7.0,272250,19446,72128.0,1009800.0,336,247,88,1439387.0,275,10.0,IntervalBudgetOptimized,Off,-2914.153846
2,2024-07-15 18:04:51.084,1,18,Monday,AdBundle,288282.0,7.0,272250,19446,72128.0,1009800.0,336,243,92,1439387.0,275,10.0,IntervalBudgetOptimized,Off,-2962.209876
3,2024-07-15 18:04:51.084,0,18,Monday,Billboard,72385.0,0.0,295120,21080,18068.0,252960.0,336,243,92,1439387.0,1190,10.0,IntervalBudgetOptimized,Off,-743.106996
4,2024-07-15 19:05:02.736,1,19,Monday,AdBundle,288282.0,0.0,272250,19446,72128.0,1009800.0,336,242,93,1439387.0,275,10.0,IntervalBudgetOptimized,Off,-2981.479339


---

## Dataset Preparation

To train a `pi_optimal` RL agent, we must first load and preprocess the ad dataset. The `pi_optimal` package provides a custom dataset class that streamlines the preprocessing pipeline. Below are the key parameters that need to be defined during this process:

- **Unit Index**:  
   This parameter, `unit_index`, identifies distinct units in the dataset. In our case, each unit corresponds to unique adset (`unit_index` column).

- **Time Column**:  
   The time column (`timestep_column`) establishes the temporal sequence of data points, enabling the model to learn from historical trends. For instance, the RL agent can consider the previous 24 hours of data (set by the `lookback_timesteps` parameter) to make informed decisions. In our case we would use the datetime column `created_at`.

- **Reward Column**:  
   The `reward_column` specifies the target that the agent seeks to optimize. Here, we previously calculated and added the `reward` column to the dataset, which reflects just the distance between the current total impressions and the expected current impression given the progression of the campaign.

- **State Columns**:  
   The state columns capture the system's current status, it includes all variables that influence campaigns delivery performance. Relevant examples include:  
   - `adset_targetings_total_population`  
   - `adset_settings_total_budget`  
   - `day_of_week`  
   - ...

   These features help the agent assess the current environment and predict outcomes effectively.

- **Action Columns**:  
   The action columns represent controllable variables, such as the CPM set or the frequency capping settingy. As you can see there are multiple action which we try to optimize at the same time.

By carefully defining these parameters, we ensure that the RL agent can interpret the dataset's structure, learn from past patterns, and make optimized decisions to optimize adset delivery.

### Configuration Options

When preparing the dataset for training, you can use either a dictionary or a list with target names to configure the dataset. This flexibility allows you to define the dataset structure and preprocessing steps in a way that best suits your needs.

- **Dictionary Configuration**: 
    Using a dictionary allows for detailed configuration of each feature, including specifying the type, processor, and evaluation metric.

- **List Configuration**: 
    Alternatively, you can use a list of target names to quickly specify the features without detailed configuration. This approach is simpler and faster for straightforward use cases.

Beides the columns mentioned above, we could also adjust the number of lookback timesteps for predicting hwo the campaign will evolve in the future. In our case we consider the last 12 hours of data to make a decision.

#### List Configuration

In [6]:
state_cols = [
    'hour_of_day',
    'day_of_week',
    'adset_name',
    'adset_impressions_diff',
    'adset_settings_total_budget',
    'adset_settings_daily_budget',
    'adset_settings_maximum_daily_impressions',
    'adset_settings_maximum_lifetime_impressions',
    'adset_settings_runtime_in_hours',
    'adset_settings_remaining_hours',
    'adset_settings_running_hours',
    'adset_targetings_total_population',
]

action_cols = [
    'adset_settings_maximum_cpm',
    'adset_targetings_frequency_capping_requests',
    'adset_settings_bidding_strategy',
    'adset_settings_pacing_type',
]

reward_col = 'reward'

timestamp_col = 'created_at'

unit_col = 'unit_index'

In [7]:
import pi_optimal as po

LOOKBACK_TIMESTEPS = 8
historical_dataset = po.datasets.timeseries_dataset.TimeseriesDataset(df=df_historical_adset_control,
                                                                      state_columns=state_cols,
                                                                      action_columns=action_cols,
                                                                      reward_column=reward_col,
                                                                      timestep_column=timestamp_col,
                                                                      unit_index=unit_col,
                                                                      lookback_timesteps=LOOKBACK_TIMESTEPS)

#### Dictionary Configuration

In [6]:
dataset_config = {
    "episode_column": "unit_index",
    "timestep_column": "created_at",
    "states": {
        0: {
            "name": "hour_of_day",
            "type": "numerical",
            "processor": {"name": "StandardScaler"},
            "evaluation_metric": "mae",
            "feature_begin_idx": 0,
            "feature_end_idx": 1
        },
        1: {
            "name": "day_of_week",
            "type": "categorial",
            "processor": {"name": "OrdinalEncoder"},
            "evaluation_metric": "accuracy",
            "feature_begin_idx": 1,
            "feature_end_idx": 2
        },
        2: {
            "name": "adset_name",
            "type": "categorial",
            "processor": {"name": "OrdinalEncoder"},
            "evaluation_metric": "accuracy",
            "feature_begin_idx": 2,
            "feature_end_idx": 3
        },
        3: {
            "name": "adset_impressions_diff",
            "type": "numerical",
            "processor": {"name": "StandardScaler"},
            "evaluation_metric": "mae",
            "feature_begin_idx": 3,
            "feature_end_idx": 4
        },
        4: {
            "name": "adset_settings_total_budget",
            "type": "numerical",
            "processor": {"name": "StandardScaler"},
            "evaluation_metric": "mae",
            "feature_begin_idx": 4,
            "feature_end_idx": 5
        },
        5: {
            "name": "adset_settings_daily_budget",
            "type": "numerical",
            "processor": {"name": "StandardScaler"},
            "evaluation_metric": "mae",
            "feature_begin_idx": 5,
            "feature_end_idx": 6
        },
        6: {
            "name": "adset_settings_maximum_daily_impressions",
            "type": "numerical",
            "processor": {"name": "StandardScaler"},
            "evaluation_metric": "mae",
            "feature_begin_idx": 6,
            "feature_end_idx": 7
        },
        7: {
            "name": "adset_settings_maximum_lifetime_impressions",
            "type": "numerical",
            "processor": {"name": "StandardScaler"},
            "evaluation_metric": "mae",
            "feature_begin_idx": 7,
            "feature_end_idx": 8
        },
        8: {
            "name": "adset_settings_runtime_in_hours",
            "type": "numerical",
            "processor": {"name": "StandardScaler"},
            "evaluation_metric": "mae",
            "feature_begin_idx": 8,
            "feature_end_idx": 9
        },
        9: {
            "name": "adset_settings_remaining_hours",
            "type": "numerical",
            "processor": {"name": "StandardScaler"},
            "evaluation_metric": "mae",
            "feature_begin_idx": 9,
            "feature_end_idx": 10
        },
        10: {
            "name": "adset_settings_running_hours",
            "type": "numerical",
            "processor": {"name": "StandardScaler"},
            "evaluation_metric": "mae",
            "feature_begin_idx": 10,
            "feature_end_idx": 11
        },
        11: {
            "name": "adset_targetings_total_population",
            "type": "numerical",
            "processor": {"name": "StandardScaler"},
            "evaluation_metric": "mae",
            "feature_begin_idx": 11,
            "feature_end_idx": 12
        },
        12: {
            "name": "reward",
            "type": "numerical",
            "processor": {"name": "StandardScaler"},
            "evaluation_metric": "mae",
            "feature_begin_idx": 12,
            "feature_end_idx": 13
        }
    },
    "actions": {
        0: {
            "name": "adset_settings_maximum_cpm",
            "type": "numerical",
            "processor": {"name": "StandardScaler"},
            "evaluation_metric": "mae",
            "feature_begin_idx": 0,
            "feature_end_idx": 1
        },
        1: {
            "name": "adset_targetings_frequency_capping_requests",
            "type": "numerical",
            "processor": {"name": "StandardScaler"},
            "evaluation_metric": "mae",
            "feature_begin_idx": 1,
            "feature_end_idx": 2
        },
        2: {
            "name": "adset_settings_bidding_strategy",
            "type": "categorial",
            "processor": {"name": "OrdinalEncoder"},
            "evaluation_metric": "accuracy",
            "feature_begin_idx": 2,
            "feature_end_idx": 3
        },
        3: {
            "name": "adset_settings_pacing_type",
            "type": "categorial",
            "processor": {"name": "OrdinalEncoder"},
            "evaluation_metric": "accuracy",
            "feature_begin_idx": 3,
            "feature_end_idx": 4
        }
    },
    "lookback_timesteps": 8,
    "reward_column": "reward",
    "states_size": 13,
    "reward_feature_idx": 12,
    "reward_vector_idx": 12,
    "actions_size": 4
}

In [7]:
import pi_optimal as po

historical_dataset = po.datasets.timeseries_dataset.TimeseriesDataset(df=df_historical_adset_control,
                                                                      dataset_config=dataset_config)

## Agent Initialization and Training

With the dataset prepared, the next crucial step is to initialize and train the Reinforcement Learning (RL) agent. This agent will learn to model the dynamics of the regional online advertising market, enabling it to make informed decisions based on the environment's behavior.

If we want to specify which models that we want to use with which parameters, we can pass these parameters to our Agent class in a list. Number of models can be increased simply by adding more model configuration to the list. If the model configuration is specified, then the default models will be neural networks.

In [8]:
from pi_optimal.agents.agent import Agent

agent = Agent()

model_config = [
    {
        "model_type": "NeuralNetwork",
        "params": {
            "hidden_layer_sizes": (32), 
            "learning_rate_init": 0.01
        }
    },
    {
        "model_type": "RandomForest",
        "params": {
            "n_estimators": 20, 
            "max_depth": 10
        }
    }
]

agent.train(historical_dataset, model_config=model_config)

Training models...:   0%|          | 0/13 [00:00<?, ?it/s]

Training models...:   0%|          | 0/13 [00:00<?, ?it/s]

### Optional Save and Load


Saving and loading agents can be done easily with the respective functions. If a name and a directory is not specified, models and configurations willbe automatically saved under agents/pi_optimal_agent.

In [9]:
agent.save()

In [10]:
test = Agent.load('agents/pi_optimal_agent')

## Evaluation and Action Prediction

After training the Reinforcement Learning (RL) agent, the next step is to evaluate its performance on new, unseen data. This involves loading data of a running campaign, preparing it using the same preprocessing pipeline as the historical dataset, and then using the trained agent to predict the optimal actions (i.e., adjustments to maxCPM, frequency capping, or bidding strategy) to optmize the delivery objective.


### Load Current Data

In [11]:
# Load the current adset control data
df_current_adset_control = pd.read_csv('data/current_adset_control.csv', parse_dates=['created_at'])

# Apply the reward calculation
df_current_adset_control["reward"] = df_current_adset_control.apply(calculate_reward, axis=1)

### Create Current Dataset

In [12]:
current_dataset = po.datasets.timeseries_dataset.TimeseriesDataset(df=df_current_adset_control,
                                                                    dataset_config=agent.dataset_config,
                                                                    train_processors=False,
                                                                    is_inference=True)

### Predict Optimal Actions

In [13]:
best_actions = agent.predict(current_dataset, inverse_transform=True, n_iter=5)

Simulating trajectories:   0%|          | 0/4 [00:00<?, ?it/s]

Simulating trajectories:   0%|          | 0/4 [00:00<?, ?it/s]

Simulating trajectories:   0%|          | 0/4 [00:00<?, ?it/s]

Simulating trajectories:   0%|          | 0/4 [00:00<?, ?it/s]

Simulating trajectories:   0%|          | 0/4 [00:00<?, ?it/s]

---

## Interpreting the Results

The agent provides a sequence of optimal actions for the time horizon. Here we print them:

In [14]:
for i in range(len(best_actions)):
    print(f"Timestep {i}:")
    print("Maximum CPM:", best_actions[i][0])
    print("Frequency Capping:", best_actions[i][1])
    print("Bidding Strategy:", best_actions[i][2])
    print("Pacing Type:", best_actions[i][3])
    print()
    print("--------------------")
    print()

Timestep 0:
Maximum CPM: 1093.697301149356
Frequency Capping: 36.55035624081035
Bidding Strategy: Maxprice
Pacing Type: EvenlyOverRuntime

--------------------

Timestep 1:
Maximum CPM: 976.5153756714803
Frequency Capping: 33.018441865149406
Bidding Strategy: Maxprice
Pacing Type: EvenlyOverRuntime

--------------------

Timestep 2:
Maximum CPM: 824.6648564289474
Frequency Capping: 20.372741176131093
Bidding Strategy: IntervalBudgetOptimized
Pacing Type: EvenlyOverRuntime

--------------------

Timestep 3:
Maximum CPM: 623.0476289630387
Frequency Capping: 16.387525339758778
Bidding Strategy: IntervalBudgetOptimized
Pacing Type: EvenlyOverRuntime

--------------------



### Multi-Step Planning

The agent optimizes actions by considering future outcomes over a multi-step horizon. This allows for efficient and forward-thinking decision-making.

### Decision-Making Options

1. **Full Application of Recommended Actions**: We could choose to apply all recommended actions immediately, for example adjusting maximum cpm, frequency capping, bidding strategy or pacing according to the agent's suggestions for the entire time horizon (e.g., the next 4 hours). Therefore the campaign will be controled based on the agent’s full plan.

2. **Incremental Application**: Alternatively, we might apply only the first action in the sequence for the next hour and then the next hour re-run the agent to generate updated recommendations. This method provides flexibility by allowing adjustments based on real-time conditions, while still leveraging the agent’s ability to look multiple steps ahead.

--- 

## Visualization

`pi_optimal` includes a **trajectory visualizer** for the simulated optimal trajectory. This tool allows you to explore the agent's recommendations and analyze their effects on energy consumption and indoor temperature over time. It provides valuable insights into the agent's behavior and helps evaluate its performance across various scenarios.

In [15]:
from pi_optimal.utils.trajectory_visualizer import TrajectoryVisualizer

trajectory_visualizer = TrajectoryVisualizer(agent, current_dataset, best_actions=best_actions)
trajectory_visualizer.display()

Simulating trajectories:   0%|          | 0/4 [00:00<?, ?it/s]

Simulating trajectories:   0%|          | 0/4 [00:00<?, ?it/s]

Simulating trajectories:   0%|          | 0/4 [00:00<?, ?it/s]

Simulating trajectories:   0%|          | 0/4 [00:00<?, ?it/s]

---

## Conclusion

This notebook demonstrates how `pi_optimal` could be used to optimize ad delivery for regional online advertising campaigns. By training a Reinforcement Learning (RL) agent on historical ad data, we can automate and optimize ad campaigns, ensuring smooth and efficient ad delivery over time. The agent learns to make informed decisions based on the environment's behavior, enabling it to maximize the desired outcome and improve resource utilization.


### Key Highlights

- **Dataset Preparation**: We loaded and preprocessed the ad dataset, defining key parameters such as unit index, time column, reward column, state columns, and action columns.
- **Data Type Agnostic Actions**: `pi_optimal` supports a wide range of actions, including continuous, discrete, and multi-dimensional actions. This flexibility allows you to model complex systems and optimize a variety of objectives. Especially in the context of ad tech, where multiple actions can be taken to optimize delivery.

---

## Next Steps

1. **Fine-Tuning the Reward Function**: Experiment with different reward functions to further optimize ad delivery performance. For example, you could consider additional factors such as budget utilization, audience targeting, or campaign reach.

2. **Hyperparameter Tuning**: Optimize the agent's hyperparameters to enhance its performance and generalization capabilities. You could explore different neural network architectures, learning rates, or optimization algorithms to improve the agent's learning efficiency.

3. **Real-Time Decision-Making**: Extend the agent's capabilities to support real-time decision-making. By integrating the agent with live data streams, you can optimize ad delivery in dynamic and rapidly changing environments.

### References

- `pi_optimal` Documentation: [GitHub](https://github.com/pi-optimal/pi_optimal)